# 네이버 뉴스 기사 크롤링
- 1) "코로나" 키워드로 검색 후 검색결과를 크롤링해서 기사링크 리스트를 만든다.
    - max page = 100을 설정해서, 하루당 1000건의 뉴스 (=100페이지*한 페이지당 10건의 뉴스)를 크롤링한다
- 2) 크롤링한 기사들의 네이버 뉴스 링크로 다시 크롤링해서 들어간다
    - 네이버 뉴스에서 제공하는 기사들이므로 같은 html 구조를 가지고 있어서, 같은 크롤링함수를 적용가능하다
    - 각 뉴스 기사에서 타이틀, 발행일, 기사 내용을 크롤링한다
    
참고: https://bumcrush.tistory.com/155

### 네이버의 API로 데이터를 받아올 때의 주의사항

- 검색 API를 사용
- 검색 결과로부터 최대 1000개까지의 결과만 가져옴
- 뉴스기사의 본문을 전부 가져오지는 못함. 대략 초입 3줄 정도?
- 일 최대 25,000번의 호출 가능
- 한 번 API 호출시 최대 100개의 결과값 수집 가능
- 최대로 수집해봤자 일부 중복되는 결과 발생


출처: https://signing.tistory.com/76 [끄적거림]

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import os

os.makedirs('./crawler', exist_ok=True)
os.chdir('./crawler')

In [2]:
os.getcwd()

'/Users/jeongjaewon/Desktop/1조/crawler'

In [3]:
def link_crawler(maxpage, query, date):
    '''
    1) 코로나 키워드로 검색한 결과에서 기사들의 네이버 뉴스 링크를 크롤링하는 함수
    '''
    import os
    os.makedirs('./files', exist_ok=True)
    
    date_ = date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1 
    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + date + "&de=" + date + "&nso=so%3Ar%2Cp%3Afrom" + date_ + "to" + date_ + "%2Ca%3A&start=" + str(page)
    url_link_list=[] #코로나 검색결과를 저장하는 리스트

    while page < maxpage_t:
        if page//100>0 and page%100==1:
            print(f'{page}page', end=' - ')
        # get requests
        try:
            request=requests.get(url)
        except:
            print(f'Connection Error: {requests.status_code}')
        #get content and parse
        content = request.content
        soup = BeautifulSoup(content, 'html.parser')
        #get link from each of the news sub box
        for sub_bx in soup.find_all("li", class_="sub_bx"):
            try:
                link = sub_bx.find("a")['href']
                url_link_list.append(link)
            except:
                pass
        page += 10
    return url_link_list

In [4]:
def write_file(url_link_list):   
    '''
    link_crawler 함수의 결과를 저장하는 함수
    '''
    with open('./files/url_link.txt', 'w') as f:
        for link in url_link_list:
            f.write(link+'\n')
    print('url writing Done')

### 이슈발생1
requests 라이브러리를 그냥 쓴다고 손쉽게 HTTP 호출을 할 수 있는 것은 아니다. 무분별한 크롤링을 막기 위해 이런 저런 장치가 되어 있어서, 가급적 브라우저를 통한 접속과 구분할 수 없게 일종의 위장이 필요하다.



출처: https://butnotforme.tistory.com/entry/python으로-업무-자동화까지-8-requests3?category=932590 

In [5]:
def news_crawler(url_link_list):
    '''
    2) 크롤링한 기사들의 네이버 뉴스 링크로 하나씩 들어가서 뉴스 기사(제목, 발행일, 내용)를 크롤링하는 함수
    '''
    from tqdm import tqdm

    news_title=[]
    published_date=[]
    news_text=[]
    #이슈 발생1 참고
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36" }

    for link in tqdm(url_link_list):
        try:
            r = requests.get(link, headers=headers)
        except:
            print('Connection Error')
        if r:
            try:
                soup = BeautifulSoup(r.content, 'html.parser')
                title = soup.select('h3#articleTitle')[0].text
                news_title.append(title)
                pdate = soup.select('.t11')[0].get_text()[:11]
                published_date.append(pdate)
                text = soup.select('#articleBodyContents')[0].get_text().replace('\n', " ").strip()
                text = text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
                news_text.append(text.strip())
            except:
                pass 

    assert len(news_title)==len(published_date)==len(news_text)
    result = pd.DataFrame({'title': news_title, 'publish':published_date, 'news_text':news_text})

    return result

### 이슈 발생2
- 기간 별로 뉴스를 긁어올때, maxpage안에서만 긁어오게 되어있음
- 이떄 정렬 기준이 최신순이기 때문에 03.10~03.03 순으로 정렬되어 긁어오게 됨
- 그러나 해당 기간의 뉴스가 너무 많이 나와 max page=100으로는 03.10에 발행된 뉴스를 긁어오는 것 밖에 안됨
- 또한, 임의로 max page를 정해서 긁어오게 된다면, 날짜 별로 다른 갯수의 기사를 긁어오게됨
- 예를 들어, 03.10일에는 500개(전체 발행기사), 03.09일에는 400개(전체 발행기사), 03.08일에는 100개
- 이는 표본 바이어스 문제가 발생하게 된다.
- **따라서, 각 날짜별로 각각 max page=100으로 설정해서 긁어오도록 한다**

In [6]:
def check_time(func):
    import time
    def wrapper(*args, **kwargs):
        st=time.time()
        res = func(*args, **kwargs)
        print(round(time.time()-st, 2),'초 소요')
        return res
    return wrapper

In [7]:
@check_time
def main_crawler(maxpage, query, s_date, e_date, save=True):
    '''
    link_crawler와 news_crawler를 동작하게 하는 메인 크롤러 함수
    '''
    from datetime import datetime, timedelta    
    
    data = pd.DataFrame()
    url_link_list = []
    
    start=datetime.strptime(s_date, '%Y-%m-%d')
    end=datetime.strptime(e_date, '%Y-%m-%d')
    period=(end-start).days
    
    #이슈발생2 참고
    for i in range(period+1):
        date = start + timedelta(i)
        date = date.strftime('%Y.%m.%d')
        print(date, 'news link crawling')
        url_link_list.extend(link_crawler(maxpage,query,date))
        if save:
            write_file(url_link_list)
      
    result=news_crawler(url_link_list)
    data = pd.concat([data,result], axis=0)
    
    return data

첫번째 주기 [2020-03-03 ~ 2020-03-10]
두번째 주기 [2020-08-15 ~ 2020-09-02]
세번째 주기 [2020-12-15 ~ 2021-01-10]

In [8]:
def main(maxpage, s_date,e_date):
    maxpage, query, s_date, e_date = maxpage, '코로나', s_date, e_date
    data = main_crawler(maxpage, query, s_date, e_date)
    return data

In [26]:
#maxpage =2로 설정해서 확인해봄
data = main(2, '2020-03-03', '2020-03-10')
data.head()

2020.03.03 news link crawling
url writing Done
2020.03.04 news link crawling
url writing Done
2020.03.05 news link crawling
url writing Done
2020.03.06 news link crawling
url writing Done
2020.03.07 news link crawling
url writing Done
2020.03.08 news link crawling
url writing Done
2020.03.09 news link crawling
url writing Done
2020.03.10 news link crawling


  1%|          | 1/119 [00:00<00:22,  5.15it/s]

url writing Done


100%|██████████| 119/119 [00:22<00:00,  5.26it/s]

24.53 초 소요


,title,publish,news_text
0,코로나19 확진 374명 추가·총 5천186명…43일만에 5천명 넘어,2020.03.03.,3일 0시 기준 대구·경북 총 4천286명·신천지 관련 총 2천698명오늘도 분주한...
1,"코로나19로 일자리 없는 제주 中불법체류자들 ""차라리 귀국""(종합)",2020.03.03.,"법무부 자진출국 유도 지난달 230명 이상 출국신고, 3일 250여명 몰려…역대 최..."
2,코로나19 무서워…꼭꼭 숨었던 불법체류자 '탈'제주,2020.03.03.,코로나19 국내 확산세로 불법체류 중국인 자진출국 잇따라[제주CBS 고상현 기자]코...
3,"""나 돌아갈래"" 코로나19로 제주 中불법체류자 수백명 탈출 행렬",2020.03.03.,"법무부 자진출국 유도 지난달 230명 이상 출국신고, 3일 하루 200여명 몰려 코..."
4,“일자리 없어 다시 가요” 코로나 여파 제주 떠나는 불법체류자들,2020.03.03.,제주출입국·외국인청에 중국인 불법체류자 출국 신청 봇물3일 제주출입국외국인청 건물 ...


In [9]:
main(2, '2020-03-03', '2020-03-10').to_csv('./files/temp.csv', index=None)

2020.03.03 news link crawling
url writing Done
2020.03.04 news link crawling
url writing Done
2020.03.05 news link crawling
url writing Done
2020.03.06 news link crawling
url writing Done
2020.03.07 news link crawling
url writing Done
2020.03.08 news link crawling
url writing Done
2020.03.09 news link crawling
url writing Done
2020.03.10 news link crawling


  0%|          | 0/120 [00:00<?, ?it/s]

url writing Done


100%|██████████| 120/120 [00:28<00:00,  4.18it/s]

30.55 초 소요


### 참고
- 각 날짜별로 크로링된 기사수는 다를 수 있음(모든 뉴스기사를 네이버 뉴스에서 제공하지 않기 때문임)
- 예를 들어, 네이버 연예에서 제공하는 뉴스는 네이버 뉴스와 html 구조가 다르므로, 이럴 경우 pass함

# 주의!! 아래의 코드를 실행하면 크롤링이 시작됩니다.
# 굉장히 오랜 시간(약 3-4시간) 이 걸리므로, 주의하세요!

In [ ]:
main(100, '2020-03-03', '2020-03-10').to_csv('./files/1st.csv')
main(100, '2020-08-15', '2020-09-02').to_csv('./files/2nd.csv', index=None)
main(100, '2020-12-15', '2020-01-10').to_csv('./files/3rd.csv', index=None)